http://ytscrapper-env.eba-7kfsymye.us-east-1.elasticbeanstalk.com/ #YT Scrapper deployed on AWS

In [23]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from json import loads


url = 'https://www.youtube.com/@PW-Foundation/videos'

# Get the html by get method
r = requests.get(url)
print(r)

# Create BeautifulSoup object
soup = BeautifulSoup(r.text, 'html.parser')
all_script_tags = soup.findAll('script')

<Response [200]>


In [24]:
def script_tag_to_json(tags: list) -> dict:
    for tag in reversed(tags):
        text: str = tag.text
        if 'ytInitialData = {"responseContext"' in text:
            return loads(text[20:-1])

    raise ValueError('Required script tag not found in the given tags.')

In [26]:
data = script_tag_to_json(all_script_tags)

#Return data from videos
def get_contents_dict(data):
    return data['contents']['twoColumnBrowseResultsRenderer']['tabs'][1]['tabRenderer']['content']['richGridRenderer']['contents']

Q1. Write a python program to extract the video URL of the first five videos.

In [27]:
def get_videoUrl(data:dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append('https://www.youtube.com/watch?v=' +
                      contents[i]['richItemRenderer']['content']['videoRenderer']['videoId'])

    return result

get_videoUrl(data)

['https://www.youtube.com/watch?v=9jfTn4lcLu4',
 'https://www.youtube.com/watch?v=c_pKfD4UgFg',
 'https://www.youtube.com/watch?v=RWcwMNVdp9c',
 'https://www.youtube.com/watch?v=jzclZf2NaFM',
 'https://www.youtube.com/watch?v=hzwEXoQDNCs']

Q2. Write a python program to extract the URL of the video thumbnails of the first five videos.

In [28]:
def get_thumbnails(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['thumbnail']['thumbnails'][-1]['url'])

    return result

get_thumbnails(data)

['https://i.ytimg.com/vi/9jfTn4lcLu4/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLDpc_pU3Upoo_CmLWDkK5RMVXrXkQ',
 'https://i.ytimg.com/vi/c_pKfD4UgFg/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLBUL9blkwGnj4aO3G-5kSIYCD0s7A',
 'https://i.ytimg.com/vi/RWcwMNVdp9c/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAdURzM_o_-lOZ4mGqGf-jb0j0TCg',
 'https://i.ytimg.com/vi/jzclZf2NaFM/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAHSbvdOVBsRHgIPVvxkmsr3h6Mqw',
 'https://i.ytimg.com/vi/hzwEXoQDNCs/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCJDqd7rC-hAV4_PfiauSCm9gcrag']

Q3. Write a python program to extract the title of the first five videos.

In [30]:
def get_title(data: dict, n:int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['title']['runs'][-1]['text'])

    return result

get_title(data)

['तो चलिए शुरू करते है AIIMS Jane Ki Tayyari !! Revealing Arjuna NEET Faculties 🙏 Check Description.',
 'Ruman Ashraf - Bihar Board Class 10th TOPPER from PW !! Udaan Batch⚡',
 'Complete Roadmap on How to Start Class 9th 🔥 || Full Year Long Strategy For Students 💯',
 'Class 10th Over 😎 अब Class 11th JEE की तैयारी !! Check Description.',
 'Aarambh Batch 2024 | For CBSE Class 11th Commerce | Biggest Launch🔥']

Q4. Write a python program to extract the number of views of the first five videos.

In [31]:
def get_viwes(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(int(contents[i]['richItemRenderer']['content']['videoRenderer']['viewCountText']['simpleText']
                      [:-6].replace(',', '')))

    return result

get_viwes(data)


[46745, 63678, 32452, 65388, 44188]

Q5. Write a python program to extract the time of posting of video for the first five videos.

In [32]:
def get_time_of_posting(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['publishedTimeText']['simpleText'])

    return result

get_time_of_posting(data)

['1 day ago', '1 day ago', '2 days ago', '3 days ago', '6 days ago']


Csv

In [34]:
def get_channel_video_details(data: dict, n: int):
    thumbnails = get_thumbnails(data, n)
    time_of_posting = get_time_of_posting(data, n)
    titles = get_title(data, n)
    video_urls = get_videoUrl(data, n)

    main_data = list(zip(video_urls, titles, thumbnails, time_of_posting))
    
    df = DataFrame.from_dict(main_data)
    df.rename(
        columns={
            0: 'video_urls',
            1: 'title',
            2: 'thumbnail_url',
            3: 'time_of_posting'
        }, inplace=True)

    return df

channel_data = get_channel_video_details(data, 30)
channel_data.to_csv('PW-Foundation.csv', index=False)